# Density map with cross validation

In [ ]:
using DIVAnd
using Makie, CairoMakie
using Statistics
using DelimitedFiles
using LinearAlgebra
using Random
using JupyterFormatter
enable_autoformat()

## Data

In [ ]:
NX = 200
NY = 200
# Box size
LX = 10.0
LY = 10.0
# Bounding box
xleft = -5.0
ybot = -5.0
xright = xleft + LX
ytop = ybot + LY
# For the synthetic case, number of points
NP = 2000
Random.seed!(1234)
xo = randn(NP)
yo = randn(NP)
inflation = ones(Float64, NP)
#test for weights on poins

# Eliminate points out of the box
sel = (xo .> xleft) .& (xo .< xright) .& (yo .> ybot) .& (yo .< ytop)

xo = xo[sel]
yo = yo[sel]
inflation = inflation[sel]
NP = size(xo)[1]

dx = LX / (NX)
dy = LY / (NY)

xg = xleft+dx/2:dx:xleft+LX
yg = ybot+dy/2:dy:ybot+LY
# for pyplot
xp = xleft:dx:xleft+LX
yp = ybot:dy:ybot+LY
maskp, (pmp, pnp), (xip, yip) = DIVAnd.DIVAnd_rectdom(xp, yp)
mask, (pm, pn), (xi, yi) = DIVAnd.DIVAnd_rectdom(xg, yg);

### Function to display the results

In [ ]:
function plot_density(dens2, title, showobs = true)
    fig = Figure()
    ax = Axis(fig[1, 1], aspect = 1, title = title)
    hm = heatmap!(ax, xip[:, 1], yip[1, :], dens2)#, colormap=Reverse(:RdYlBu))
    if showobs
        scatter!(ax, xo, yo, color = :white, markersize = 3)
    end
    Colorbar(fig[1, 2], hm)
    fig
end

In [ ]:
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0)
plot_density(dens2, "Automatic uniform scale selection\n$(LHM)")

In [ ]:
LHMIN = 0.28 * 0.6
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, LHMIN)
plot_density(dens2, "Automatic uniform scale selection\n$(LHM)", false)

## Cross validation

In [ ]:
gaussiansol=1/(2*pi)*exp.(-0.5*(xi.^2.0)-0.5*(yi.^2.0))

LS=collect(0.01:0.005:.3)
NSP=size(LS)[1]

CV1=zeros(Float64,NSP)
CV2=zeros(Float64,NSP)
CVex=zeros(Float64,NSP)
for i=1:NSP
    dens2,LHM,LCV,LSCV=DIVAnd_heatmap(mask,(pm,pn),(xi,yi),(xo,yo),inflation,LS[i])
    CV1[i]=LCV
    CV2[i]=LSCV
    CVex[i]=norm(gaussiansol-dens2)
end
@show findmax(CV1), findmin(CV2),findmin(CVex)

In [ ]:
fig = Figure(size = (900, 400))
ax1 = Axis(fig[1, 1])
lines!(ax1, LS, CV1)
ax2 = Axis(fig[1, 2])
lines!(ax2, LS, CV2)
ax3 = Axis(fig[1, 3])
lines!(ax3, LS, CVex)
display(fig)
@show LS[findmax(CV1)[2]], LS[findmin(CV2)[2]], LS[findmin(CVex)[2]]

In [ ]:
# Ok factor 0.62 seems correct. Now try with inflation?

In [ ]:
0.284 * 0.62

In [ ]:
NX = 200
NY = 200
# Box size
LX = 10.0
LY = 10.0
# Bounding box
xleft = -5.0
ybot = -5.0
xright = xleft + LX
ytop = ybot + LY
# For the synthetic case, number of points
NP = 200000
nmax = 15000
Random.seed!(1234)
xo = randn(NP)
yo = randn(NP)
inflation = ones(Float64, NP)
#test for weights on poins

# Eliminate points out of the box
sel = (xo .> xleft) .& (xo .< xright) .& (yo .> ybot) .& (yo .< ytop)

xo = xo[sel]
yo = yo[sel]
inflation = inflation[sel]

NP = size(xo)[1]

dx = LX / (NX)
dy = LY / (NY)

xg = xleft+dx/2:dx:xleft+LX
yg = ybot+dy/2:dy:ybot+LY
# for pyplot
xp = xleft:dx:xleft+LX
yp = ybot:dy:ybot+LY
maskp, (pmp, pnp), (xip, yip) = DIVAnd.DIVAnd_rectdom(xp, yp)

mask, (pm, pn), (xi, yi) = DIVAnd.DIVAnd_rectdom(xg, yg)

dens2, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0; nmax = nmax)

In [ ]:
plot_density(dens2, "Automatic uniform scale selection\n$(LHM)")

In [ ]:
figure()
pcolor(xip,yip,dens2),colorbar()
scatter(xo,yo,s=2,c="white")
title("Automatic uniform scale selection $(LHM)")

gaussiansol=1/(2*pi)*exp.(-0.5*(xi.^2.0)-0.5*(yi.^2.0))

LS=collect(0.03:0.005:.3)
NSP=size(LS)[1]

CV1=zeros(Float64,NSP)
CV2=zeros(Float64,NSP)
CVex=zeros(Float64,NSP)
for i=1:NSP
    dens2,LHM,LCV,LSCV=DIVAnd_heatmap(mask,(pm,pn),(xi,yi),(xo,yo),inflation,LS[i];nmax=nmax)
    CV1[i]=LCV
    CV2[i]=LSCV
    CVex[i]=norm(gaussiansol-dens2)
end

In [ ]:
@show findmax(CV1), findmin(CV2), findmin(CVex)
fig = Figure(size = (900, 400))
ax1 = Axis(fig[1, 1])
lines!(ax1, LS, CV1)
ax2 = Axis(fig[1, 2])
lines!(ax2, LS, CV2)
ax3 = Axis(fig[1, 3])
lines!(ax3, LS, CVex)
display(fig)
@show LS[findmax(CV1)[2]], LS[findmin(CV2)[2]], LS[findmin(CVex)[2]]

In [ ]:
0.13 * 0.62

In [ ]:
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask,
    (pm, pn),
    (xi, yi),
    (xo, yo),
    inflation,
    LS[findmin(CVex)[2]];
    nmax = nmax,
)
plot_density(dens2, "", false)

In [ ]:
dens2, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0.13; nmax = nmax)
plot_density(dens2, "", false)

In [ ]:
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask,
    (pm, pn),
    (xi, yi),
    (xo, yo),
    inflation,
    LS[findmin(CV2)[2]];
    nmax = nmax,
)
plot_density(dens2, "", false)

In [ ]:
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask,
    (pm, pn),
    (xi, yi),
    (xo, yo),
    inflation,
    LS[findmax(CV1)[2]];
    nmax = nmax,
)
plot_density(dens2, "", false)